In [1]:
!pip install numpy
!pip install scikit-surprise
import os
import pandas as pd
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import KFold
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from surprise.accuracy import rmse
from collections import defaultdict
from surprise import KNNBasic
from surprise import SVD
from surprise import SVDpp
from surprise import NMF
from surprise import accuracy
from surprise import AlgoBase

In [2]:

# Function to load the ratings and movies datasets

# dir = '/content/drive/MyDrive/CSE573-SWM-Movie-Recommender/DATA'
def loadDataset():
  ratings_df = pd.read_csv('./Data/ratings.csv')
  movies_df = pd.read_csv('./Data/movies.csv')
  
  ratings_df.drop('timestamp',axis =1, inplace = True)

  return ratings_df,movies_df


In [3]:
# Calculates the evaluations metrics Precision and Recall for each loop of cross validation

def metricsAtK(predictions, k=10, threshold=3.5):

    # dictionary to store user and the respective predicted ratings of the movies
    user_predicted = defaultdict(list)
    for u_id, _, rating_actual, rating_predicted, _ in predictions:
        user_predicted[u_id].append((rating_predicted, rating_actual))

    precisions = dict()
    recalls = dict()

    for u_id, u_ratings in user_predicted.items():
        u_ratings.sort(key=lambda x: x[0], reverse=True)

        # Calculating actual values of the ratings
        actual = sum((ratings_true >= threshold) for (_, ratings_true) in u_ratings)
        # Calculating predicted values of the ratings
        predicted = sum((estimate_value >= threshold) for (estimate_value, _) in u_ratings[:k])

        # Calculating true positives and negatives
        positive_true = sum(((ratings_true >= threshold) and (estimate_value >= threshold))
                              for (estimate_value, ratings_true) in u_ratings[:k])
        negative_true = sum(((ratings_true < threshold) and (estimate_value < threshold))
                              for (estimate_value, ratings_true) in u_ratings[:k])
        
        # Calculating actual and predicted positives and negatives
        positive_predicted = predicted if predicted != 0 else 1
        negative_predicted = predicted if predicted != 1 else 0
        positive_actual = actual if actual != 0 else 1
        negative_actual = actual if actual != 1 else 0

        # Calculating precision, recall and accuracy
        precisions[u_id] = positive_true / positive_predicted
        recalls[u_id] = positive_true / positive_actual

    return precisions, recalls

In [4]:
def fit(model, ratings_data, num_of_splits=5):
  # Generator object for k - cross validation
  kf = KFold(n_splits=num_of_splits)
  split_df = list()

  i = 1
  for train, test in kf.split(ratings_data):
      predictions = model.fit(train).test(test)
      rmse = accuracy.rmse(predictions, verbose=False)
      mae = accuracy.mae(predictions, verbose=False)
      precisions, recalls = metricsAtK(predictions, k=5, threshold=4)
      precision = sum(prec for prec in precisions.values()) / len(precisions)
      recall = sum(rec for rec in recalls.values()) / len(recalls)
      f1_score = (2*precision*recall)/(precision+recall)
      split_df.append([i,precision,recall,rmse,f1_score,mae])
      i +=1

  split_df = pd.DataFrame(split_df, columns=['Split', 'Precision', 'Recall', 'RMSE', 'F1 Score', 'MAE'])
  return split_df

In [5]:
def getSortedPredictions(predictions):
    
    sorted_predictions = defaultdict(list)    
    for u_id, id, _, rating_predicted, _ in predictions:
        sorted_predictions[u_id].append((id, rating_predicted))

    for u_id, u_ratings in sorted_predictions.items():
        u_ratings.sort(key=lambda x: x[1], reverse=True)

    return sorted_predictions

In [6]:
def inference(model):
  trainset = ratings_data.build_full_trainset()

  # build_anti_testset will generate the entires for the movies which the user has not rated. i.e 
  # The completement of the user's ratings. It assumes the rating to be equal to the global mean of the ratings.
  testset = trainset.build_anti_testset() 

  # Training the model with trainset and getting predictions using the generated testset
  model.fit(trainset)
  predictions = model.test(testset)

  return predictions

In [7]:
# Post processing the predictions to produce the list of recommendations
def getRecommendations(predictions, n=10):

  # Getting sorted predictions
  total = getSortedPredictions(predictions)

  # Extracting only n number of movie predictions
  for user_id, user_ratings in total.items():
      total[user_id] = user_ratings[:n]

  total_df = pd.DataFrame.from_dict(total)
  total_df = total_df.transpose()

  result = []
  for user_id,user_ratings in total.items():
    result.append(total_df.loc[user_id])

  #Developing recommendations
  recommendations = []
  for i in result:
    recommended_movieIds=[]
    for x in range(0, n):
      recommended_movieIds.append(i[x][0])
    recommendations.append(recommended_movieIds)

  recommendation_list = []
  for i in recommendations:
    df = movies_df[movies_df['movieId'].isin(i)]
    temp = df['title'].tolist()
    recommendation_list.append(temp)

  recommendation_df = pd.DataFrame(recommendation_list)
  return recommendation_df

## Loading Dataset

In [8]:
ratings_df , movies_df = loadDataset()

In [9]:
reader = Reader(rating_scale=(0.5, 5.0))
ratings_keys = ['userId', 'movieId', 'rating']
ratings_filter = ratings_df[ratings_keys]
ratings_data = Dataset.load_from_df(ratings_filter, reader)

## Modeling Selection

## SVD

In [10]:
msd_knn = SVD(n_factors=30, n_epochs=20, lr_all=0.008, reg_all=0.08)
#msd_preds = msd_knn.fit(trainset).test(testset)


In [11]:
# Generating evaluation metrics Precision, Recall, F1-Score, RMSE, MAE for the ratings dataset
msd_metrics = fit(msd_knn, ratings_data)
msd_metrics

,Split,Precision,Recall,RMSE,F1 Score,MAE
0,1,0.558579,0.219046,0.863196,0.314688,0.662733
1,2,0.567459,0.229017,0.864921,0.326332,0.663175
2,3,0.553968,0.229794,0.869022,0.324840,0.667769
3,4,0.557143,0.234674,0.860797,0.330245,0.662138
4,5,0.533745,0.211811,0.867367,0.303272,0.666370


In [12]:
msd_predictions = inference(msd_knn)

In [13]:
msd_df = getRecommendations(msd_predictions)
msd_df.to_csv('msd_knn.csv',index = False)

### Using Pearson Correlation

In [14]:
# Defining the similarity options with pearson correlation.
sim_options = {
    'name': 'pearson'
}

pearson_knn = KNNBasic(k= 35, n_epochs=25,sim_options = sim_options)
#cosine_preds = cosine_knn.fit(trainset).test(testset)


In [15]:
# Generating evaluation metrics Precision, Recall, F1-Score, RMSE, MAE for the ratings dataset
pearson_metrics = fit(pearson_knn, ratings_data)
pearson_metrics

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.


,Split,Precision,Recall,RMSE,F1 Score,MAE
0,1,0.681500,0.261927,0.975192,0.378415,0.753310
1,2,0.674945,0.258220,0.978945,0.373534,0.754511
2,3,0.654098,0.251986,0.982151,0.363815,0.758051
3,4,0.648960,0.251235,0.974332,0.362236,0.750020
4,5,0.642857,0.245741,0.964915,0.355563,0.744201


In [16]:

pearson_predictions = inference(pearson_knn)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [17]:

pearson_df = getRecommendations(pearson_predictions)
pearson_df.to_csv('pearson_knn.csv',index = False)

### Using Cosine similarity

In [18]:
# Defining the similarity options with cosine similarity.
sim_options = {
    'name': 'cosine'
}

cosine_knn = KNNBasic(k= 35, n_epochs=25)
#cosine_preds = cosine_knn.fit(trainset).test(testset)


In [19]:
# Generating evaluation metrics Precision, Recall, F1-Score, RMSE, MAE for the ratings dataset
cosine_metrics = fit(cosine_knn, ratings_data)
cosine_metrics

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


,Split,Precision,Recall,RMSE,F1 Score,MAE
0,1,0.673852,0.266666,0.950745,0.382116,0.729525
1,2,0.652244,0.263808,0.954977,0.375671,0.729906
2,3,0.693634,0.266490,0.940647,0.385048,0.719891
3,4,0.660902,0.263820,0.951512,0.377106,0.727735
4,5,0.669235,0.273755,0.942311,0.388564,0.722014


In [20]:
cosine_predictions = inference(cosine_knn)


Computing the msd similarity matrix...
Done computing similarity matrix.


In [21]:
cosine_df = getRecommendations(cosine_predictions)
cosine_df.to_csv('cosine_knn.csv',index = False)
